# Loading the MobileNet Model for Finetuning
## Freezing all layers except for the top 4 layers.

In [11]:
from keras.applications.mobilenet import MobileNet

img_shape = {'row':224, 'col':224}

mnet = MobileNet(weights = 'imagenet', include_top = False, input_shape=(img_shape['row'], img_shape['col'],3))

for layer in mnet.layers:
    layer.trainable = False

for (i,layer) in enumerate(mnet.layers):
    print(str(i)+ " " + layer.__class__.__name__, layer.trainable)



17235968/17225924 [==============================] - 4s 0us/step
0 InputLayer False
1 Conv2D False
2 BatchNormalization False
3 ReLU False
4 DepthwiseConv2D False
5 BatchNormalization False
6 ReLU False
7 Conv2D False
8 BatchNormalization False
9 ReLU False
10 ZeroPadding2D False
11 DepthwiseConv2D False
12 BatchNormalization False
13 ReLU False
14 Conv2D False
15 BatchNormalization False
16 ReLU False
17 DepthwiseConv2D False
18 BatchNormalization False
19 ReLU False
20 Conv2D False
21 BatchNormalization False
22 ReLU False
23 ZeroPadding2D False
24 DepthwiseConv2D False
25 BatchNormalization False
26 ReLU False
27 Conv2D False
28 BatchNormalization False
29 ReLU False
30 DepthwiseConv2D False
31 BatchNormalization False
32 ReLU False
33 Conv2D False
34 BatchNormalization False
35 ReLU False
36 ZeroPadding2D False
37 DepthwiseConv2D False
38 BatchNormalization False
39 ReLU False
40 Conv2D False
41 BatchNormalization False
42 ReLU False
43 DepthwiseConv2D False
44 BatchNormalization F

In [12]:
def addTopModelMobileNet(bottom_model, num_classes):
    """Creates the head of the model to be placed on the top of the bottom layers
    
    Args:
        bottom_model (model): The frozen layers
        num_classes(int): No. of categories to be classified into
    Return:
        top_model: model head that does the classification
    
    """

    top_model = bottom_model.output
    top_model = GlobalAveragePooling2D()(top_model)
    top_model = Dense(1024, activation = 'relu')(top_model)
    top_model = Dense(1024, activation = 'relu')(top_model)
    top_model = Dense(512, activation = 'relu')(top_model)
    top_model = Dense(num_classes, activation = 'softmax')(top_model)
    return top_model

In [13]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, GlobalAveragePooling2D
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import batch_normalization
from keras.models import Model

num_classes = 10

FC_Head = addTopModelMobileNet(mnet, num_classes)
model = Model(inputs = mnet.input, outputs = FC_Head)

print(model.summary())


Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 112, 112, 32)      864       
                                                                 
 conv1_bn (BatchNormalizatio  (None, 112, 112, 32)     128       
 n)                                                              
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)     288       
                                                                 
 conv_dw_1_bn (BatchNormaliz  (None, 112, 112, 32)     128       
 ation)                                                    

In [14]:
from keras.preprocessing.image import ImageDataGenerator

train_data_dir = "./monkey_breed/train/"
val_data_dir = "./monkey_breed/validation/"

train_datagen = ImageDataGenerator(rescale=1./255,rotation_range=45,width_shift_range=0.3,height_shift_range=0.3,horizontal_flip=True,fill_mode='nearest')
validation_datagen = ImageDataGenerator(rescale=1./255)

batch_size = 32

train_generator = train_datagen.flow_from_directory(train_data_dir,target_size=(img_shape['row'], img_shape['col']),batch_size=batch_size,class_mode='categorical')
val_generator = validation_datagen.flow_from_directory(val_data_dir,target_size=(img_shape['row'], img_shape['col']),batch_size=batch_size,class_mode='categorical')

Found 1098 images belonging to 10 classes.
Found 272 images belonging to 10 classes.


In [20]:
from keras.optimizers import rmsprop_v2
from keras.callbacks import ModelCheckpoint, EarlyStopping

optimizer = rmsprop_v2.RMSProp(learning_rate=0.001)

checkpoint = ModelCheckpoint("./monkey_breed_mobileNet.h5", monitor='val_loss', save_best_only = True, verbose = 1)

earlystop = EarlyStopping(monitor = 'val_loss', min_delta = 0, patience = 3, verbose = 1, restore_best_weights = True)

callbacks = [earlystop, checkpoint]

model.compile(loss='categorical_crossentropy',optimizer=optimizer, metrics=['accuracy'])

nb_train_samples = 1098
nb_validation = 272

epochs = 5

history = model.fit_generator(train_generator,steps_per_epoch = nb_train_samples//batch_size,epochs = epochs,callbacks = callbacks, validation_data=val_generator,validation_steps=nb_validation//batch_size)

C:\Users\adeda\AppData\Local\Temp\ipykernel_8224\1584540778.py:19: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(train_generator,steps_per_epoch = nb_train_samples//batch_size,epochs = epochs,callbacks = callbacks, validation_data=val_generator,validation_steps=nb_validation//batch_size)


Epoch 1/5
34/34 [==============================] - ETA: 0s - loss: 1.1669 - accuracy: 0.8424
Epoch 1: val_loss improved from inf to 0.17461, saving model to .\monkey_breed_mobileNet.h5
34/34 [==============================] - 36s 995ms/step - loss: 1.1669 - accuracy: 0.8424 - val_loss: 0.1746 - val_accuracy: 0.9570
Epoch 2/5
34/34 [==============================] - ETA: 0s - loss: 0.5928 - accuracy: 0.8752
Epoch 2: val_loss improved from 0.17461 to 0.17052, saving model to .\monkey_breed_mobileNet.h5
34/34 [==============================] - 33s 964ms/step - loss: 0.5928 - accuracy: 0.8752 - val_loss: 0.1705 - val_accuracy: 0.9648
Epoch 3/5
34/34 [==============================] - ETA: 0s - loss: 0.2796 - accuracy: 0.9250
Epoch 3: val_loss did not improve from 0.17052
34/34 [==============================] - 32s 929ms/step - loss: 0.2796 - accuracy: 0.9250 - val_loss: 0.2335 - val_accuracy: 0.9453
Epoch 4/5
34/34 [==============================] - ETA: 0s - loss: 0.3041 - accuracy: 0.91

In [21]:
from keras.models import load_model

classifier = load_model('./monkey_breed_mobileNet.h5')

In [22]:
import os
import cv2
import numpy as np
from os import listdir
from os.path import isfile, join

monkey_breeds_dict = {"[0]":"mantled_howler",
                    "[1]":"patas_monkey",
                    "[2]":"bald_uakari",
                    "[3]":"japanese_macaque",
                    "[4]":"pygmy_marmoset",
                    "[5]":"white_headed_capuchin",
                    "[6]":"silver_marmoset",
                    "[7]":"common_squirrel_monkey",
                    "[8]":"black_headed_night_monkey",
                    "[9]":"nilgiri_langur"
                    }
monkey_breeds_dict_n = {"n0": "mantled_howler ", 
                    "n1": "patas_monkey",
                    "n2": "bald_uakari",
                    "n3": "japanese_macaque",
                    "n4": "pygmy_marmoset ",
                    "n5": "white_headed_capuchin",
                    "n6": "silvery_marmoset",
                    "n7": "common_squirrel_monkey",
                    "n8": "black_headed_night_monkey",
                    "n9": "nilgiri_langur"}

def draw_test(name, pred, im):
    monkey = monkey_breeds_dict[str(pred)]
    black = [0,0,0]
    expanded_image = cv2.copyMakeBorder(im, 80, 0, 0, 100 , cv2.BORDER_CONSTANT, value=black)
    cv2.putText(expanded_image, monkey, (20,60), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
    cv2.imshow(name, expanded_image)

def getRandomImage(path):
    folders = list(filter(lambda x: os.path.isdir(join(path,x)), listdir(path)))
    random_dir = np.random.randint(0, len(folders))
    path_class = folders[random_dir]
    print("class - " + monkey_breeds_dict_n[str(path_class)])
    file_path = path + path_class
    file_names = [f for f in listdir(file_path) if  isfile(join(file_path, f))]
    random_file_index = np.random.randint(0, len(file_names))
    image_name = file_names[random_file_index]
    return cv2.imread(file_path+'/'+image_name)

for i in range(0,10):
    input_im = getRandomImage("./monkey_breed/validation/")
    input_original = input_im.copy()
    input_original = cv2.resize(input_original, None, fx=0.5, fy=0.5, interpolation = cv2.INTER_LINEAR)
    
    input_im = cv2.resize(input_im, (224, 224), interpolation = cv2.INTER_LINEAR)
    input_im = input_im / 255.
    input_im = input_im.reshape(1,224,224,3) 
    
    # Get Prediction
    res = np.argmax(classifier.predict(input_im, 1, verbose = 0), axis=1)
    
    # Show image with predicted class
    draw_test("Prediction", res, input_original) 
    cv2.waitKey(0)

cv2.destroyAllWindows()

class - pygmy_marmoset 
class - patas_monkey
class - pygmy_marmoset 
class - japanese_macaque
class - patas_monkey
class - black_headed_night_monkey
class - black_headed_night_monkey
class - nilgiri_langur
class - bald_uakari
class - pygmy_marmoset 
